Restricted Boltzman Machines
=================

A [Restricted Boltzmann machine (RBM)](https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine) is a generative stochastic artificial neural network that can learn a probability distribution over its set of inputs. They are a particular form of [Boltzman Machines](https://en.wikipedia.org/wiki/Boltzmann_machine) subject to a **restriction**. The restriction is that there are no connections between nodes within a group of units (meaning that the network form a bipartite graph, see below).

The RBM is made of two layers, each one having a certain number of units. The input units are called **visible units** of the RBM because their states are observed. The feature detectors correspond to non-observed **hidden units**. The hidden units are often referred to as **latent variables**, as they do not result directly from the observed data. The two layers are connected through a matrix of weights. The units inside the layer are not connected, meaning that the network form a [bipartite graph](https://en.wikipedia.org/wiki/Bipartite_graph).

<p align="center">
<img src="../etc/img/rbm_architecture.png" width="400">
</p>


When RBMs are viewed as neural networks they are used as function approximators that can map the observations to the expectations of the latent variables in the top layer. The top layer can be interpreted as the learned features which can serve as inputs for a supervised learning system.

However, the main application of RBMs is as **generative models**.  Learning an RBM corresponds to fitting its parameters such that the RBM approximates the distribution underlying the training data. After learning, the RBM can be used to generate new samples. For instance, we can fix some visible units corresponding to a partial observation and sample the remaining visible units to complete the observation.

A question arise: is it possible to use RBMs for **classification**? There are some tricks that allows a RBM to be used in this way. One of the most interesting is to extend the input variable using an **indicator variable** as follows:

<p align="center">
<img src="../etc/img/rbm_labeled_data.png" width="500">
</p>

The input image of a digit is extended using a graphical $5 \times 2$ matrix, where the location related to the number is marked in black. In this way we can train the RBM on the marked training set, and then test the classification capability on the unmarked test set. If the RBM correctly modelled the data distribution it will be able to return the marked version of the test image. Unfortunately this trick cannot be used so easily, because it implies to find a way to include the indicator variable in the training set and to easily read its value in the returned sample.



Energy of a configuration
-----------------------------

The visible and hidden units are often organised as vectors, and a pair of visible-hidden vectors is called a **configuration**. A joint configuration of the visible and hidden units has an **energy** (see Hopfield, 1982) given by:

$$E(v,h) = -a^{T}v -b^{T}h -v^{T} Wh $$

where the matrix of weights $W$ (size $m \times n$) associated with the connection between hidden unit $h$ and visible unit $v$, as well as bias weights $a$ for the visible units and $b$ for the hidden units.
This definition of Energy is the same used in [Hopefield Networks](https://en.wikipedia.org/wiki/Hopfield_network).


Joint probability distribution over units
------------------------------------------------

Using the energy it is possible to define a series of probability distribution of both visible and hidden units. Here we consider both $v$ and $h$ as Bernoulli random variables, meaning that they are binary units (howevever there are ways to generalize to real units).

The [joint probability](https://en.wikipedia.org/wiki/Joint_probability_distribution) of every possible pair of a visible and a hidden vector can be defined as follows:

$$ P(v,h) = \frac{1}{Z} e^{-E(v,h)}$$

where the **partition function** $Z$ is a normalisation factor, given by summing over all possible pairs of visible and hidden vectors:

$$ Z = \sum_{v,h} e^{-E(v,h)}$$

An important problem arises when the estimation of $Z$ is required. Since both $v$ and $h$ are binary there is an exponential number of values that they can take. It is possible to understand this point considering $v$ and $h$ as bit arrays. If the array has $8$ elements then there are $2^8$ possible combination to consider. It is intuitive to understand that increasing the number of bits makes the number of possible combination explode (since there is an exponential factor involved). This problem makes the estimation of the partition function **intractable**. The intractability of $Z$ can be managed using different forms of approximation (introduced later).

Marginal probability distributions over units
---------------------------------------------------

Using the energy it is also possible to compute the [marginal probability distribution](https://en.wikipedia.org/wiki/Marginal_distribution) on the visible units:

$$ P(v) = \frac{1}{Z} \sum_{h} e^{-E(v,h)}$$

and on the hidden units:

$$ P(h) = \frac{1}{Z} \sum_{v} e^{-E(v,h)}$$

Since both $P(v)$ and $P(h)$ require the partition function, we cannot directly estimate them.

It is possible to show that the marginal distribution of $P(v)$, showed above, can be rearranged and expressed in another form:

$$ P(v) = \frac{1}{Z} \ \text{exp} \big[ a^{T} v + \sum_{i=1}^{H} \text{log} \big( 1 + \text{exp} ( b_{j} + W_{[j,:]} v) \big) \big] $$

where $H$ is the total number of hidden units and $W_{[j,:]}$ means we are accessing the $j$-th row of $W$. Here I used $\text{exp}()$ insted of $e$ only to make the notation less cluttered. You must notice that the expression $\text{log}(1+\text{exp}())$ is also known as the [softplus function](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) that is a smooth version of the ReLu activation function. We can rewrite $p(x)$ based on this observation:

$$ P(v) = \frac{1}{Z} \ \text{exp} \big[ a^{T} v + \sum_{i=1}^{H} \text{sofplus} ( b_{i} + W_{[j,:]} v) \big) \big] $$

Now it is useful to analyse the last equation and try to understand when the **probability** $P(x)$ **get larger**. A first possible way to increase the probability is to increse the outcome of the product $a^{T} v$. Let's recall that $a$ is a vector of bias terms, and $v$ is a vector of binary units. Another way to increase the probability is to increase the term inside the softmax function. One way to do it is to increase the bias vector $b$. Another way is to make the visible vector $v$ well alligned with the rows of $W$.

It turns out that in literature the content of the exponential function above is often called the **free energy** $F()$ and for this reason we can simply use the notation:

$$ P(v) = \frac{1}{Z} \ \text{exp} \big[ -F(x) \big] $$


Conditional probability distributions (inference)
-------------------------------------------------------

Using the energy we can also obtain the [conditional probability distribution](https://en.wikipedia.org/wiki/Conditional_probability_distribution) of the $m$ visible units given the hidden units:


$$ P(v|h) = \prod_{i=1}^{m} P(v_{i} | h) $$

and the of the $n$ hidden units with respect to the visible units:

$$ P(h|v) = \prod_{j=1}^{n} P(h_{j} | v) $$

In both cases we used the assumption that the visible units are independent given the hidden units (and viceversa). Differently from the joint and marginal probability distributions, the conditional distribution is **tractable** because there is not any partition function involved in both $P(v|h)$ and $P(h|v)$.

It is interesting to see how we can obtain $P(v|h)$ for each unit $i$, using a matrix notation:


$$ P(v_{i}|h) = \text{sig}(a_{i} + h^{T} W_{[:,i]}) $$

where $\text{sig}()$ is the sigmoid function and $W_{[:,i]}$ means that we are accessing the $i$-th column of $W$. The same thing can be done when we need to estimate $P(h|v)$:

$$ P(h_{j}|v) = \text{sig}(b_{j} + W_{[j,:]} v) $$

where $W_{[j,:]}$ mean we are accessing the $j$-th row of $W$.

Gibbs sampling
----------------

Gibbs sampling is a simple **Markov Chain Monte Carlo (MCMC)** method for producing samples from the **joint probability distribution** of multiple random variables. The idea is to construct a Markov chain by updating each variable given the state of the others (conditional distribution).


We said that due to the absence of connections between variables of the same layer, the conditional distributions $P(h | v)$ and $P(v |h)$ factorize nicely. This conditional independence makes Gibbs sampling especially easy. We do not need to sample a value for $h_{j}$ with respect to the other hidden units, because they are not connected. This mean that the states of all variables in one layer can be sampled jointly in just two steps: 

1. sampling a new state $h$ for the hidden neurons based on $P(h | v)$
2. sampling a state $v$ for the visible layer based on $P(v |h)$

This is also referred to as **block Gibbs sampling**.

RBM as an undirected graphical model (Markov Random Field)
-------------------------------------------------------------

An RBM can be described in term of graphical models, in particular in terms of undirected graphs such as **Markov Random Fields (MRFs)**. A RBM is an MRF


Let's recall here that in an undirected graph the vertices, two vertices are said to be **adjacent** if they are connected by an edge. A **clique** is a subset of vertices of an undirected graph such that every two distinct vertices in the clique are adjacent.



Training objective
---------------------



Contrastive divergence
-------------------------

To solve the problem we can use a method proposed by Hinton and called **Contrastive Divergence (CD)**.


Resources
------------

- *"A Practical Guide to Training RBMs"*, Geoffrey Hinton [[pdf]](https://www.cs.toronto.edu/~hinton/absps/guideTR.pdf)
- *"Notes on Contrastive Divergence"*, Woodford [[pdf]](http://www.robots.ox.ac.uk/~ojw/files/NotesOnCD.pdf)
- *"Training Products of Experts by Minimizing Contrastive Divergence"*, 2002, Geoffrey Hinton [[web]](https://www.mitpressjournals.org/doi/abs/10.1162/089976602760128018)
- *"Training Restricted Boltzmann Machines: An Introduction"*, Fisher and Igel [[pdf]](https://pdfs.semanticscholar.org/e1a1/c3d3e0336d6bfc48fd480a9d18a4f92edc60.pdf)